In [27]:
!pip install yfinance pandas matplotlib requests plotly ipywidgets --quiet

import os
import json
import textwrap
import datetime as dt

import requests
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# In Colab this may or may not help with widgets; harmless elsewhere
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except Exception:
    pass

%matplotlib inline

SEC_HEADERS = {
    "User-Agent": "Your Name your.email@example.com",
    "Accept-Encoding": "gzip, deflate"
}

def find_symbol_from_company_name(company_name: str) -> str | None:
    """
    Find symbol by company name using SEC official company list.
    """
    url = "https://www.sec.gov/files/company_tickers.json"
    resp = requests.get(url, headers=SEC_HEADERS)
    resp.raise_for_status()
    data = resp.json()

    company_upper = company_name.upper()
    best_match = None

    for entry in data.values():
        name = entry["title"].upper()
        if company_upper in name:
            best_match = entry["ticker"]
            break

    return best_match

# ---- TEST / INPUT ----
company_name = "Apple"
ticker_symbol = find_symbol_from_company_name(company_name)
print("Found symbol:", ticker_symbol)


def download_price_history(symbol: str, period: str = "10y"):
    """
    Download up to 10 years of historical data for a symbol using yfinance.
    Returns a DataFrame or None if it fails.
    """
    try:
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period=period, auto_adjust=False)
    except Exception as e:
        print("Error downloading history:", e)
        return None

    if hist is None or hist.empty:
        print("No history returned for", symbol)
        return None

    hist.index = pd.to_datetime(hist.index)
    return hist


def plot_time_ranges_interactive(hist: pd.DataFrame, symbol: str):
    """
    Plot interactive Plotly charts for:
      - 1 Day, 1 Week, 1 Month, 1 Year, 2 Years, 3 Years, 5 Years, 10 Years.

    For each chart:
      - Title includes % change over the full range (Δ +X.XX%).
      - Hover tooltip shows Open, High, Low, Close, Volume.
      - Zoom / pan via toolbar + range slider.
      - 'Reset View' button resets axes to the original full range.
    """
    hist = hist.copy()
    end = hist.index.max()

    def slice_range(days):
        if days is None:
            return hist
        start = end - pd.Timedelta(days=days)
        return hist.loc[hist.index >= start]

    ranges = {
        "1 Day": 1,
        "1 Week": 7,
        "1 Month": 30,
        "1 Year": 365,
        "2 Years": 365 * 2,
        "3 Years": 365 * 3,
        "5 Years": 365 * 5,
        "10 Years": None,  # full history
    }

    for label, days in ranges.items():
        sliced = slice_range(days)
        if sliced.empty:
            print(f"{label}: Not enough data to plot.")
            continue

        # Ensure OHLCV exists
        required_cols = ["Open", "High", "Low", "Close", "Volume"]
        if not all(col in sliced.columns for col in required_cols):
            print(f"{label}: Missing OHLCV columns in data.")
            continue

        close_series = sliced["Close"].dropna()
        if close_series.empty:
            print(f"{label}: No valid close prices.")
            continue

        # Full-range % change (Close)
        start_price = close_series.iloc[0]
        end_price = close_series.iloc[-1]
        if pd.notna(start_price) and pd.notna(end_price) and start_price != 0:
            pct_change_full = (end_price / start_price - 1) * 100
            pct_text_full = f"{pct_change_full:+.2f}%"
        else:
            pct_text_full = "N/A"

        title = f"{symbol} – Close Price – {label} (Δ {pct_text_full})"

        # Add OHLCV to hover via customdata
        customdata = sliced[["Open", "High", "Low", "Close", "Volume"]].values

        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=sliced.index,
                y=sliced["Close"],
                mode="lines",
                name=symbol,
                customdata=customdata,
                hovertemplate=(
                    "Date: %{x}<br>"
                    "Open: %{customdata[0]:.2f}<br>"
                    "High: %{customdata[1]:.2f}<br>"
                    "Low: %{customdata[2]:.2f}<br>"
                    "Close: %{customdata[3]:.2f}<br>"
                    "Volume: %{customdata[4]:,.0f}<extra></extra>"
                ),
            )
        )

        fig.update_layout(
            title=title,
            xaxis_title="Date",
            yaxis_title="Price",
            hovermode="x unified",
            height=400,
            # Range slider at the bottom for easier zoom
            xaxis=dict(
                rangeslider=dict(visible=True),
                type="date"
            ),
            updatemenus=[
                dict(
                    type="buttons",
                    direction="left",
                    buttons=[
                        dict(
                            label="Reset View",
                            method="relayout",
                            args=[{"xaxis.autorange": True, "yaxis.autorange": True}],
                        )
                    ],
                    x=1,
                    xanchor="right",
                    y=1.15,
                    yanchor="top",
                    showactive=False,
                )
            ],
        )

        print(f"\n=== {label} ===")
        print("Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.")
        print("Click 'Reset View' on the chart to return to full range.")
        fig.show()


def percent_change_between_dates(hist: pd.DataFrame, start_date: str, end_date: str):
    """
    Compute percentage change in Close price between two dates.

    Parameters:
      hist       : full history DataFrame (e.g. hist_10y)
      start_date : string like '2022-01-01'
      end_date   : string like '2023-03-15'
    """
    if hist is None or hist.empty:
        print("History is empty.")
        return

    hist_sorted = hist.sort_index()

    # Convert date strings to timestamps
    start_ts = pd.to_datetime(start_date)
    end_ts = pd.to_datetime(end_date)

    try:
        # Get nearest available indices for the requested dates
        start_idx = hist_sorted.index.get_loc(start_ts, method="nearest")
        end_idx = hist_sorted.index.get_loc(end_ts, method="nearest")
    except Exception as e:
        print("Error locating dates:", e)
        return

    # Ensure start_idx <= end_idx
    if start_idx > end_idx:
        start_idx, end_idx = end_idx, start_idx

    start_row = hist_sorted.iloc[start_idx]
    end_row = hist_sorted.iloc[end_idx]

    start_price = start_row["Close"]
    end_price = end_row["Close"]

    if pd.isna(start_price) or pd.isna(end_price) or start_price == 0:
        print("Cannot compute percentage change due to missing/invalid prices.")
        return

    pct = (end_price / start_price - 1) * 100

    print(f"From {hist_sorted.index[start_idx].date()} (Close={start_price:.2f}) "
          f"to {hist_sorted.index[end_idx].date()} (Close={end_price:.2f})")
    print(f"→ Percentage change: {pct:+.2f}%")


# ---- Run the charts for the current ticker ----
if 'ticker_symbol' not in globals() or ticker_symbol is None:
    print("Ticker symbol not found. Go back and adjust company_name.")
else:
    hist_10y = download_price_history(ticker_symbol, period="10y")
    if hist_10y is not None:
        plot_time_ranges_interactive(hist_10y, ticker_symbol)
    else:
        print("Could not download history for ticker:", ticker_symbol)


Found symbol: AAPL

=== 1 Day ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.



=== 1 Week ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.



=== 1 Month ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.



=== 1 Year ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.



=== 2 Years ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.



=== 3 Years ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.



=== 5 Years ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.



=== 10 Years ===
Hover to see OHLCV. Use Zoom/Box Zoom and the range slider.
Click 'Reset View' on the chart to return to full range.
